# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [1]:
from os import getcwd
from numpy import exp
from scipy.optimize import minimize
from ticktack import ode

So for just modelling the troposphere I will want to set things up so that I have a differential equation where $f(x) = e^{x^{8}}$ is the rate of change. I will then use `scipy.optimize.minimize()` to minimize a loss function with respect to the solution of the ode as determined by the `return` of `ode.odeint`. 

In [2]:
def dydx(t, *args):
    peak, span, height = args  # Loading parameters
    return height * exp(-((t - peak) / (span / 2)) ** 8)   # Gaussian pulse

In [ ]:
def loss(params=()):
    